# Transform

In [1]:
import torch
import torch.nn as nn

from torchhk.transform import *

## 1. Change One Layer

In [2]:
a = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=4)
b = transform_layer(a, nn.Conv2d, nn.ConvTranspose2d, 
                    args={"in_channels" : ".in_channels", "out_channels" : 16, "kernel_size" : 5}, 
                    attrs={"weight" : ".weight"})

print("--- Before ---")
print("Layer :", a)
print("Layer.weight :", a.weight[0, 0, 0])
print("Layer.bias :", a.bias[0])

print("--- After ---")
print("Layer :", b)
print("Layer.weight :", b.weight[0, 0, 0])
print("Layer.bias :", b.bias[0])

--- Before ---
Layer : Conv2d(3, 8, kernel_size=(4, 4), stride=(1, 1))
Layer.weight : tensor([ 0.0362, -0.0313, -0.1010,  0.0237], grad_fn=<SelectBackward>)
Layer.bias : tensor(-0.1002, grad_fn=<SelectBackward>)
--- After ---
Layer : ConvTranspose2d(3, 16, kernel_size=(5, 5), stride=(1, 1))
Layer.weight : tensor([ 0.0362, -0.0313, -0.1010,  0.0237], grad_fn=<SelectBackward>)
Layer.bias : tensor(-0.0055, grad_fn=<SelectBackward>)


## 2. Change Activation Functions of a model

### 2.1 inplace = True

In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.conv_layer = nn.Sequential(
            nn.Conv2d(1,16,5),
            nn.ReLU(),
            nn.Conv2d(16,32,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(32,64,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        
        self.fc_layer = nn.Sequential(
            nn.Linear(64*3*3,100),
            nn.ReLU(),
            nn.Linear(100,10)
        )       
        
    def forward(self,x):
        out = self.conv_layer(x)
        out = out.view(-1,64*3*3)
        out = self.fc_layer(out)

        return out

In [4]:
a = CNN()

In [5]:
b = transform_model(a, nn.ReLU, nn.LeakyReLU, inplace=True)

C:\Users\slcf\workspace\GitHub\pytorch-custom-utils\torchhk\transform.py:31: Warning: 
 * Caution : The Input Model is CHANGED because inplace=True.
  warnings.warn("\n * Caution : The Input Model is CHANGED because inplace=True.", Warning)


In [6]:
# New model returned.
b

CNN(
  (conv_layer): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): LeakyReLU(negative_slope=0.01)
    (2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
    (3): LeakyReLU(negative_slope=0.01)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
    (6): LeakyReLU(negative_slope=0.01)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_layer): Sequential(
    (0): Linear(in_features=576, out_features=100, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=100, out_features=10, bias=True)
  )
)

In [7]:
# The original model is also changed.
a 

CNN(
  (conv_layer): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): LeakyReLU(negative_slope=0.01)
    (2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
    (3): LeakyReLU(negative_slope=0.01)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
    (6): LeakyReLU(negative_slope=0.01)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_layer): Sequential(
    (0): Linear(in_features=576, out_features=100, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=100, out_features=10, bias=True)
  )
)

### 2.1 inplace = False

In [8]:
c = transform_model(a, nn.LeakyReLU, nn.ReLU, inplace=False)

In [9]:
a

CNN(
  (conv_layer): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): LeakyReLU(negative_slope=0.01)
    (2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
    (3): LeakyReLU(negative_slope=0.01)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
    (6): LeakyReLU(negative_slope=0.01)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_layer): Sequential(
    (0): Linear(in_features=576, out_features=100, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=100, out_features=10, bias=True)
  )
)

In [10]:
# Only new model is changed
c

CNN(
  (conv_layer): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_layer): Sequential(
    (0): Linear(in_features=576, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=10, bias=True)
  )
)